# Jupyter notebook based on ImageD11 to process scanning 3DXRD data
# Written by Haixing Fang, Jon Wright and James Ball
## Date: 28/03/2024

In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())
PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

In [ ]:
# import functions we need

import os
import concurrent.futures
import timeit

import matplotlib
%matplotlib widget

import h5py
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt

from ImageD11.sinograms.sinogram import GrainSinogram, build_slice_arrays, write_slice_recon, read_slice_recon, write_h5, read_h5
import ImageD11.columnfile
from ImageD11.sinograms import properties, roi_iradon
from ImageD11.blobcorrector import eiger_spatial
from ImageD11.grain import grain
from ImageD11 import cImageD11

from ImageD11.nbGui import nb_utils as utils

In [ ]:
# USER: Pass path to dataset file

dset_file = 'si_cube_test/processed/Si_cube/Si_cube_S3DXRD_nt_moves_dty/Si_cube_S3DXRD_nt_moves_dty_dataset.h5'

ds = ImageD11.sinograms.dataset.load(dset_file)
   
sample = ds.sample
dataset = ds.dsname
processed_data_root_dir = ds.analysisroot

print(ds)
print(ds.shape)

In [ ]:
minor_phase_name = "Au"

minor_phase_grains_path = os.path.splitext(ds.grainsfile)[0] + f'_{minor_phase_name}.h5'

grainsinos_Au = read_h5(minor_phase_grains_path, ds)
grains_Au = [gs.grain for gs in grainsinos_Au]

grainsinos_Fe = read_h5(ds.grainsfile, ds)
grains_Fe = [gs.grain for gs in grainsinos_Fe]

slice_arrays_Fe = read_slice_recon(ds.grainsfile)
rgb_x_array_Fe, rgb_y_array_Fe, rgb_z_array_Fe, grain_labels_array_Fe, raw_intensity_array_Fe = slice_arrays_Fe

slice_arrays_Au = read_slice_recon(minor_phase_grains_path)
rgb_x_array_Au, rgb_y_array_Au, rgb_z_array_Au, grain_labels_array_Au, raw_intensity_array_Au = slice_arrays_Au

In [ ]:
rgb_Fe = rgb_z_array_Fe.copy()
rgb_Au = rgb_z_array_Au.copy()

In [ ]:
fig, axs = plt.subplots(1, 2, constrained_layout=True, sharex=True, sharey=True)
axs[0].imshow(rgb_Fe, origin="lower")
axs[1].imshow(rgb_Au, origin="lower")
plt.show()

In [ ]:
rgb_Au_alpha = np.append(rgb_Au, np.zeros((rgb_Au.shape[0], rgb_Au.shape[1], 1)), axis=2)

In [ ]:
rgb_Au_alpha[rgb_Au_alpha[:, :, 0] != 0, 3] = 1

In [ ]:
(raw_intensity_array_Au != 0).astype(float)

In [ ]:
fig, axs = plt.subplots(1, 2, constrained_layout=True, sharex=True, sharey=True, figsize=(16, 9))
axs[0].imshow(raw_intensity_array_Fe, cmap="viridis", origin="lower")
axs[0].imshow(rgb_Au_alpha, origin="lower")
axs[1].imshow(raw_intensity_array_Fe, cmap="viridis", origin="lower")
axs[0].set_title("Fe intensity map with Aus overlaid")
axs[1].set_title("Fe intensity map")
plt.show()

In [ ]:
fig, ax = plt.subplots(constrained_layout=True, sharex=True, sharey=True)
ax.imshow(rgb_Fe, origin="lower")
ax.imshow(raw_intensity_array_Au, alpha=(raw_intensity_array_Au != 0).astype(float), cmap="grey", origin="lower")
ax.set_title("Fe IPF Z colours with white Au")
plt.show()

In [ ]:
# orientation relationship stuff

# pick a Au grain

Au_grain = grainsinos_Au[25]

# find out where it is in the sample

fig, ax = plt.subplots(constrained_layout=True, sharex=True, sharey=True)
ax.imshow(rgb_Fe, origin="lower")
ax.imshow(Au_grain.recons["iradon"], alpha=(Au_grain.recons["iradon"] > 0.05).astype(float), origin="lower")
ax.set_title("Ni IPF Z colours with single Au")

# from ImageD11.sinograms.geometry import real_space_to_recon_space

from ImageD11.sinograms.geometry import sample_to_recon

i, j = sample_to_recon(x=Au_grain.grain.translation[0], y=Au_grain.grain.translation[1], recon_shape=Au_grain.recons["iradon"].shape, ystep=ds.ystep)

print(i, j)

vr = 50

ax.set_xlim(j-vr, j+vr)
ax.set_ylim(i+vr, i-vr)
plt.show()

In [ ]:
# find Ni grain surrounding Au grain

Fe_grain_gid = grain_labels_array_Fe[np.round(i).astype(int), np.round(j).astype(int)].astype(int)

print(Fe_grain_gid)

In [ ]:
# confirm Ni grain selection

# rgb_Fe_masked = rgb_

fig, ax = plt.subplots(constrained_layout=True, sharex=True, sharey=True)
ax.imshow(grain_labels_array_Fe == Fe_grain_gid, cmap="grey", origin="lower")
ax.imshow(Au_grain.recons["iradon"], alpha=(Au_grain.recons["iradon"] > 0.05).astype(float), origin="lower")
ax.set_title("Fe IPF Z colours with single Au")

vr = 50

ax.set_xlim(j-vr, j+vr)
ax.set_ylim(i+vr, i-vr)
plt.show()

In [ ]:
# we now have a Au grain and the Fe grain that it's inside

# look for orientation relationships

Fe_grain = grainsinos_Fe[Fe_grain_gid]

print(Fe_grain.grain.U)
print(Au_grain.grain.U)

In [ ]:
# https://doi.org/10.1016/S1005-0302(12)60169-8
# we are looking for MC Aus
# 4 different types of OR present
# A {001} Au // {001} matrix , <100> Au // <100> matrix
# B [001] Au // [310] matrix , (020) Au // (1-31) matrix
# C [110] Au // [310] matrix , (-11-1) Au // (002) matrix
# D [001] Au // [001] matrix , (-260) Au // (020) matrix

# let's check A

In [ ]:
from xfab.parameters import read_par_file

par_file_Fe = os.path.join(processed_data_root_dir, 'Fe_refined.par')
par_file_Au = os.path.join(processed_data_root_dir, 'Au_refined.par')

pars_Fe = read_par_file(par_file_Fe)
pars_Au = read_par_file(par_file_Au)

In [ ]:
ucell_Fe = [pars_Fe.get("cell__a"), pars_Fe.get("cell__b"), pars_Fe.get("cell__c"), pars_Fe.get("cell_alpha"), pars_Fe.get("cell_beta"), pars_Fe.get("cell_gamma")]
ucell_Au = [pars_Au.get("cell__a"), pars_Au.get("cell__b"), pars_Au.get("cell__c"), pars_Au.get("cell_alpha"), pars_Au.get("cell_beta"), pars_Au.get("cell_gamma")]

In [ ]:
ucell_Au

In [ ]:

from diffpy.structure import Lattice, Structure
from orix.crystal_map import Phase
from orix.quaternion import Orientation, Rotation, symmetry
from orix.vector import Miller, Vector3d

struc_Fe = Structure(lattice=Lattice(*ucell_Fe))
struc_Au = Structure(lattice=Lattice(*ucell_Au))

cubic_Fe = Phase(point_group="m-3m", structure=struc_Fe)
cubic_Au = Phase(point_group="m-3m", structure=struc_Au)

# crystal frame:

matrix_plane = Miller(hkl=[2, 2, 0], phase=cubic_Fe)
Au_plane = Miller(hkl=[0, 0, 2], phase=cubic_Au)

matrix_dir = Miller(uvw=[1, 0, 0], phase=cubic_Fe)
Au_dir = Miller(hkl=[1, 0, 0], phase=cubic_Au)

# orientations in Orix are sample-to-crystal, so we have to invert
o_Fe = Orientation.from_matrix(Fe_grain.grain.U.T)
o_Au = Orientation.from_matrix(Au_grain.grain.U.T)

matrix_plane_lab = ~o_Fe * matrix_plane.symmetrise(unique=True)
Au_plane_lab = ~o_Au * Au_plane.symmetrise(unique=True)

matrix_dir_lab = ~o_Fe * matrix_dir.symmetrise(unique=True)
Au_dir_lab = ~o_Au * Au_dir.symmetrise(unique=True)

In [ ]:
fig, axs = plt.subplots(1, 2, constrained_layout=True, sharex=True, sharey=True)

axs[0].imshow(grain_labels_array_Fe == Fe_grain_gid, cmap="grey", origin="lower")
axs[0].imshow(Au_grain.recons["astra"], alpha=(Au_grain.recons["astra"] > 0.05).astype(float), origin="lower")
axs[0].set_title("Directions")

axs[1].imshow(grain_labels_array_Fe == Fe_grain_gid, cmap="grey", origin="lower")
axs[1].imshow(Au_grain.recons["astra"], alpha=(Au_grain.recons["astra"] > 0.05).astype(float), origin="lower")
axs[1].set_title("Planes")

vr = 100

axs[0].set_xlim(Au_grain.grain.translation[0]-vr, Au_grain.grain.translation[0]+vr)
axs[0].set_ylim(Au_grain.grain.translation[1]+vr, Au_grain.grain.translation[1]-vr)

ars = 5
txs = (50*5)*1/ars

for inc in range(len(Au_dir.symmetrise(unique=True).coordinates)):
    arrow_centre = Au_grain.grain.translation[0:2]
    axs[0].quiver(arrow_centre[0], arrow_centre[1], Au_dir_lab.coordinates[inc, 0], Au_dir_lab.coordinates[inc, 1], color="r", scale=ars)
    axs[0].annotate(np.array_str(Au_dir.symmetrise(unique=True).coordinates[inc], precision=None, suppress_small=True), arrow_centre + Au_dir_lab.coordinates[inc, 0:2]*[1, -1]*txs, ha="center", va="center", c="red")

ars = 5
txs = (50*5)*1/ars

for inc in range(len(matrix_dir.symmetrise(unique=True).coordinates)):
    arrow_centre = Au_grain.grain.translation[0:2] + [-20, -20]
    axs[0].quiver(arrow_centre[0], arrow_centre[1], matrix_dir_lab.coordinates[inc, 0], matrix_dir_lab.coordinates[inc, 1], color="b", scale=ars)
    axs[0].annotate(np.array_str(matrix_dir.symmetrise(unique=True).coordinates[inc], precision=None, suppress_small=True),  arrow_centre + matrix_dir_lab.coordinates[inc, 0:2]*[1, -1]*txs, ha="center", va="center", c="blue")
    
    
    
ars = 10
txs = (50*5)*1/ars

for inc in range(len(Au_dir.symmetrise(unique=True).coordinates)):
    arrow_centre = Au_grain.grain.translation[0:2]
    axs[1].quiver(arrow_centre[0], arrow_centre[1], Au_plane_lab.coordinates[inc, 0], Au_plane_lab.coordinates[inc, 1], color="r", scale=ars)
    axs[1].annotate(np.array_str(Au_plane.symmetrise(unique=True).coordinates[inc], precision=None, suppress_small=True), arrow_centre + Au_plane_lab.coordinates[inc, 0:2]*[1, -1]*txs, ha="center", va="center", c="red")

ars = 10
txs = (50*5)*1/ars

for inc in range(len(matrix_dir.symmetrise(unique=True).coordinates)):
    arrow_centre = Au_grain.grain.translation[0:2] + [-20, -20]
    axs[1].quiver(arrow_centre[0], arrow_centre[1], matrix_plane_lab.coordinates[inc, 0], matrix_plane_lab.coordinates[inc, 1], color="b", scale=ars)
    axs[1].annotate(np.array_str(matrix_plane.symmetrise(unique=True).coordinates[inc], precision=None, suppress_small=True),  arrow_centre + matrix_plane_lab.coordinates[inc, 0:2]*[1, -1]*txs, ha="center", va="center", c="blue")


plt.show()

In [ ]:
from scipy.spatial.transform import Rotation as R
import ImageD11.sym_u
cubic = ImageD11.sym_u.cubic()
for op in cubic.group:
    v = R.from_matrix((Fe_grain.grain.U.T @ op.T) @ Au_grain.grain.U).as_rotvec( degrees=True)
    ang = np.linalg.norm(v)
    print(ang, v/ang)

In [ ]:
# matrix is 1
# Au is 2

# define an orientation matrix for each of the ORs
from orix.quaternion import Misorientation, Orientation

# A {001} Au // {001} matrix , <100> Au // <100> matrix

matrix_dirs = Miller(hkl=[0, 0, 2], phase=cubic_Fe)
Au_dirs = Miller(hkl=[2, 2, 0], phase=cubic_Au)

misorien_A = Misorientation.from_align_vectors(Au_dirs, matrix_dirs)

In [ ]:


# print(misorien_B)

min_misoriens = []

for Au_grain in grains_Au[:]:
    Au_grain_pos_nearest_px = Au_grain.translation.copy()
    Au_grain_pos_nearest_px = np.round(Au_grain_pos_nearest_px).astype(int)
    Fe_grain_gid = grain_labels_array_Fe[Au_grain_pos_nearest_px[1], Au_grain_pos_nearest_px[0]].astype(int)
    try:
        Fe_grain = [grain for grain in grains_Fe if grain.gid == Fe_grain_gid][0]
        o_Fe = Orientation.from_matrix(Fe_grain.U.T, symmetry=cubic_Fe.point_group)
        o_Au = Orientation.from_matrix(Au_grain.U.T, symmetry=cubic_Au.point_group)
        
        misorien_actual = Misorientation(o_Fe * (~o_Au), symmetry=(o_Au.symmetry, o_Fe.symmetry))
        misorien_actual = misorien_actual.map_into_symmetry_reduced_zone()
        
        misorien_ref = misorien_A
        misorien_ref = misorien_ref.map_into_symmetry_reduced_zone()
        
        misorien_diff = Misorientation(misorien_ref * (~misorien_actual), symmetry=misorien_actual.symmetry)
        
        misorien_diff = misorien_diff.map_into_symmetry_reduced_zone()
        
        min_misorien = np.rad2deg(misorien_diff.angle)
        
        print(Fe_grain.gid, Au_grain.gid, min_misorien)
        
        min_misoriens.append(min_misorien)
    except IndexError:
        continue

In [ ]:
np.rad2deg(misorien_actual.angle)

In [ ]:
misorien_actual.axis

In [ ]:
o_Au.to_euler("mtex")

In [ ]:
np.savetxt(os.path.join(processed_data_root_dir, 'Fe_grain_matrix.txt'), o_Fe.to_matrix()[0])

In [ ]:
np.savetxt(os.path.join(processed_data_root_dir, 'Au_grain_matrix.txt'), o_Au.to_matrix()[0])

In [ ]:
np.savetxt(os.path.join(processed_data_root_dir, 'OR_matrix.txt'), misorien_A.to_matrix()[0])

In [ ]:
fig, ax = plt.subplots()

ax.hist(np.array(min_misoriens), bins=50)
# ax.set_xlim(0,10)

plt.show()

In [ ]:
min_misoriens